In [96]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os

import time

urlList = set()
areaList = set()

basicArea = ['jiangan', 'jianghan', 'qiaokou', 'dongxihu', 'wuchang', 'qingshan', 'hongshan', 'donghugaoxin',
             'jiangxia', 'caidian', 'huangbei', 'xinzhou','zhuankoukaifaqu']
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWekit 537.36 (KHTML, like Gecko) Chrome",
            "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}

def getSubUrl(basicArea):
    global areaList
    for area in basicArea:
        print('现在进行到' + area)
        url = 'http://wh.lianjia.com/ershoufang/' + area
        areaHtml = urlopen(url)
        bsObj = BeautifulSoup(areaHtml)
        print(url)
        aList = bsObj.select('div[data-role="ershoufang"]')[0].findAll('div')[1].findAll('a')
        for a in aList:
            if a['href'] not in areaList:
                areaList.add(a['href'][12:-1])
    with open('./lianjia/subAreaList', 'a+') as subAreaList:
        for area in areaList:
            subAreaList.write(area + '\n')

def scrapHouse(areaList):
    global urlList
    for area in areaList:
        with open('./lianjia/' + area, 'a+') as txtFile:
            basicUrl = 'https://wh.lianjia.com/ershoufang/' + area
            print('\n\n现在进行到' + area )
            pageNum = getPageNum(basicUrl)
            for num in range(1, pageNum+1):
                url = basicUrl + '/pg' + str(num)
                html = urlopen(url)
                try:
                    html = urlopen(url)
                    bsObj = BeautifulSoup(html)
                    listLen = len(bsObj.select('ul[class^="sell"]')[0].findAll('li'))
                    linkdivs = bsObj.select('div[class=title]')[1:listLen+1]
                    for div in linkdivs:
                        link = div.a['href']
                        if link not in urlList:
                            urlList.add(link)
                            txtFile.write(link + '\n')
                    print('地区:' + area + '第' + str(num) + '页爬取成功\n')
                except:
                    print('地区:' + area + '第' + str(num) + '页爬取失败\n')


def getPageNum(url):
    html = urlopen(url)
    bsObj = BeautifulSoup(html)
    num = eval(bsObj.select('div[page-data]')[0]['page-data'])['totalPage']
    return num

def cleanDataOne(dataList):
    data = list()
    for item in dataList:
        data.append(item[4:])
    return data

def cleanDataTwo(dataList):
    return dataList[1::2]

getSubUrl(basicArea)
scrapHouse(areaList)

现在进行到jiangan
http://wh.lianjia.com/ershoufang/jiangan
现在进行到jianghan
http://wh.lianjia.com/ershoufang/jianghan
现在进行到qiaokou
http://wh.lianjia.com/ershoufang/qiaokou
现在进行到dongxihu
http://wh.lianjia.com/ershoufang/dongxihu
现在进行到wuchang
http://wh.lianjia.com/ershoufang/wuchang
现在进行到qingshan
http://wh.lianjia.com/ershoufang/qingshan
现在进行到hongshan
http://wh.lianjia.com/ershoufang/hongshan
现在进行到donghugaoxin
http://wh.lianjia.com/ershoufang/donghugaoxin
现在进行到jiangxia
http://wh.lianjia.com/ershoufang/jiangxia
现在进行到caidian
http://wh.lianjia.com/ershoufang/caidian
现在进行到huangbei
http://wh.lianjia.com/ershoufang/huangbei
现在进行到xinzhou
http://wh.lianjia.com/ershoufang/xinzhou
现在进行到zhuankoukaifaqu
http://wh.lianjia.com/ershoufang/zhuankoukaifaqu


现在进行到changfengchangmatou
地区:changfengchangmatou第1页爬取成功

地区:changfengchangmatou第2页爬取成功

地区:changfengchangmatou第3页爬取成功

地区:changfengchangmatou第4页爬取成功

地区:changfengchangmatou第5页爬取成功

地区:changfengchangmatou第6页爬取成功

地区:changfengchangmatou第7页爬取成功

地区:changfengchan

In [227]:
len(urlList)

38172

In [240]:
import csv
#('房屋类型', '所在楼层', '建筑面积', '户型结构', '套内面积', '建筑类型', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '是否配备电梯', '产权年限', '挂牌时间', '交易权属', '上次交易', '房屋用途', '是否满两年', '产权所属(是否共有)', '抵押信息', '房本备件', '小区名', '首付', '税费', '挂牌价', '单价','url'))
with open('./lianjia.csv', 'a+') as csvFile:
    write = csv.writer(csvFile)
    write.writerow((u'好', '2'))
    

In [229]:
# 基本信息

def cleanData(bsObj, url):
    data = list()
    # 基本信息
    a = bsObj.select_one('div[class="base"] div[class="content"]').get_text().replace(' ', '').replace('\n\n', '').split('\n')
    # 交易属性
    b = bsObj.select_one('div[class="transaction"] div[class="content"]').get_text().replace(' ', '').replace('\n', ' ').replace('  ', ' ').replace('  ', ' ').split(' ')[1:-1][1::2]   
    # 小区位置
    c = bsObj.select_one('div[class="communityName"] a').get_text()
    # 税费
    d = bsObj.select_one('span[class="taxtext"]').get_text()
    # 总价
    e = bsObj.select_one('span[class="total"]').get_text()
    # 单价
    f = bsObj.select_one('span[class="unitPriceValue"]').get_text()
    for num in range(0, len(a)):
        if num == 2 or num == 4 or num == 11:
            data += re.findall('[0-9.]+', a[num])
        elif num == 10:
            if a[num] == '有':
                data.append(True)
            else:
                data.append(False)
        else:
            data.append(a[num][4:])
    if b[4] == '未满两年':
        b[4] = False
    else:
        b[4] = True
    if b[5] == '非共有':
        b[5] = False
    else:
        b[5] = True
    data += b
    data.append(c)
    data += re.findall('[0-9.]+',d)
    data += re.findall('[0-9.]+',e)
    data += re.findall('[0-9.]+',f)   
    numlist = [2,4,11,21,22,23,24]
    for num in numlist:
        data[num] = float(data[num])
    data.append(url)
    return data
print(cleanData(bsObj, '123'))
# 房屋类型, 所在楼层, 建筑面积, 户型结构, 套内面积, 建筑类型, 房屋朝向, 建筑结构, 装修情况, 梯户比例, 是否配备电梯, 产权年限, 挂牌时间, 交易权属, 上次交易, 房屋用途, 是否满两年, 产权所属(是否共有), 抵押信息, 房本备件, 小区名, 首付, 税费, 挂牌价, 单价      

['2室2厅1厨1卫', '低楼层(共30层)', 94.57, '平层', 75.36, '板楼', '东南', '钢混结构', '精装', '两梯四户', False, 70.0, '2018-07-14', '商品房', '2017-08-09', '普通住宅', False, False, '有抵押30万元客户偿还', '已上传房本照片', '新世界恒大华府', 65.0, 13.8, 215.0, 22735.0, '123']


1


In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/page3.html")
bsObj = BeautifulSoup(html)
print(bsObj.find("img", {"src": "../img/gifts/img1.jpg"}).parent.previous_sibling.get_text())


$15.00



In [182]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

mm = '首付65.1万 税费13.1万(仅供参考)'
ee = '22735元/平米'
re.findall("[0-9.]+", mm)

['65.1', '13.1']

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org' + articleUrl)
    bsObj = BeautifulSoup(html)
    return bsObj.find('div', {'id': 'bodyContent'}).findAll("a", href=re.compile('^(/wiki/)((?!:).)*$'))
links = getLinks("/wiki/Kevin_Bacon")
'''while len(links) > 0:
    newArticle = links[random.randint(0, len(links) - 1)].attrs['href']
    print((newArticle))
    links = getLinks(newArticle)'''

"while len(links) > 0:\n    newArticle = links[random.randint(0, len(links) - 1)].attrs['href']\n    print((newArticle))\n    links = getLinks(newArticle)"

In [4]:
 from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org" + pageUrl)
    bsObj = BeautifulSoup(html)
    for link in bsObj.findAll("a",href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print(newPage)
                getLinks(newPage)
        
#getLinks("")

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org" + pageUrl)
    bsObj = BeautifulSoup(html)
    try:
        print(bsObj.h1.get_text())
        print(bsObj.find(id = 'mw-content-text').findAll('p')[0])
        print(bsObj.find(id = 'ca-edit').find('span').find('a').attrs['href'])
    except AttributeError:
        print('页面缺少一些属性！不过不用担心')
    for link in bsObj.findAll('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print("-------------\n" + newPage)
                pages.add(newPage)
                getLinks(newPage)
#getLinks("/wiki/Kevin_Bacon")

In [2]:
# page35 about collecting via network

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

#获取页面所有内容的列表
def getInternalLinks(bsObj, includeUrl):
    internalLinks = []
    for link in bsObj.findAll("a", href=re.compile("^(/|.*" + includeUrl +")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                internalLinks.append(link.attrs['href'])
    return internalLinks

def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    for link in bsObj.findAll('a', href=re.compile("^(http|www)((?!" + excludeUrl+ ").)*$")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks
def splitAddress(address):
    addressParts = address.replace("http://", "").split("/")
    return addressParts
def getRandomExternalLinks(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html)
    externalLinks = getExternalLinks(bsObj, splitAddress(startingPage)[0])
    if len(externalLinks) == 0:
        internalLinks = getExternalLinks(startingPage)
        return getNextExternalLink(internalLinks[random,randint(0, len(internalLinks) - 1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks) - 1)]
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLinks('http://oreilly.com')
    print("随机外链是" + externalLink)
    followExternalOnly(externalLink)
#followExternalOnly("http://oreilly.com")

In [1]:
# page 81 about txt

from urllib.request import urlopen
textPage = urlopen('http://www.pythonscraping.com/pages/warandpeace/chapter1-ru.txt')
#print(str(textPage.read(), 'utf-8'))

In [11]:
# page 85 about csv

from urllib.request import urlopen
from io import StringIO
import csv

data = urlopen('http://pythonscraping.com/files/MontyPythonAlbums.csv').read().decode('ascii', 'ignore')
dataFile = StringIO(data)
csvReader = csv.reader(dataFile)

for row in csvReader:
    #print(row)
    print("The album \"" + row[0] + "\" was released in " + str(row[1]))

The album "Name" was released in Year
The album "Monty Python's Flying Circus" was released in 1970
The album "Another Monty Python Record" was released in 1971
The album "Monty Python's Previous Record" was released in 1972
The album "The Monty Python Matching Tie and Handkerchief" was released in 1973
The album "Monty Python Live at Drury Lane" was released in 1974
The album "An Album of the Soundtrack of the Trailer of the Film of Monty Python and the Holy Grail" was released in 1975
The album "Monty Python Live at City Center" was released in 1977
The album "The Monty Python Instant Record Collection" was released in 1977
The album "Monty Python's Life of Brian" was released in 1979
The album "Monty Python's Cotractual Obligation Album" was released in 1980
The album "Monty Python's The Meaning of Life" was released in 1983
The album "The Final Rip Off" was released in 1987
The album "Monty Python Sings" was released in 1989
The album "The Ultimate Monty Python Rip Off" was release

In [11]:
# page 88 about PDF
# 由于版本问题 pdfminer无法正常运行
from urllib.request import urlopen
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open

def readPDF(pdfFile):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    
    process_pdf(rsrcmgr, device, pdfFile)
    device.close()
    
    content = retstr.getvalue()
    retstr.close()
    return content

def process_pdf(rsrcmgr, device, fp, pagenos=None, maxpages=0, password='', caching=True, check_extractable=True):
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching, check_extractable=check_extractable):
        interpreter.process_page(page)
    return


#pdfFile = urlopen('http://pythonscraping.com/pages/warandpeace/chapter1.pdf')
#outputString = readPDF(pdfFile)
#print(outputString)
#pdfFile.close()

In [5]:
# page 89 about MS and .docx
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO
from bs4 import BeautifulSoup

wordFile = urlopen('http://pythonscraping.com/pages/AWordDocument.docx').read()
wordFile = BytesIO(wordFile)
document = ZipFile(wordFile)
xml_content = document.read('word/document.xml')
#print(xml_content.decode('utf-8'))

wordObj = BeautifulSoup(xml_content.decode('utf-8'))
textStrings = wordObj.findAll("w:t")
for textElem in textStrings:
    print(textElem.text)

In [3]:
import requests

params = {'firstname': 'Ryan', 'lastname': 'Mitchell'}
r = requests.post("http://pythonscraping.com/pages/files/processing.php", data=params)
print(r.text)

Hello there, Ryan Mitchell!


In [6]:
# page 133 about form and ajax

#from selenium import webdriver
#import time
#
#driver = webdriver.PhantomJS(executable_path='D:\phantomjs-2.1.1-windows\phantomjs-2.1.1-windows\bin\phantomjs.exe')
#driver.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html')
#time.sleep(3)
#print(driver.find_element_by_id('content').text)
#driver.close()

In [2]:
# sample from 简书 using headless chrome

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
#chrome_options.add_argument("--headless")

base_url = "http://www.baidu.com/"
#对应的chromedriver的放置目录
driver = webdriver.Chrome(executable_path=(r'D:\chromedriver.exe'), chrome_options=chrome_options)

driver.get(base_url + "/")

start_time=time.time()
print('this is start_time ',start_time)

driver.find_element_by_id("kw").send_keys("selenium webdriver")
driver.find_element_by_id("su").click()
driver.save_screenshot('screen.png')

driver.close()

end_time=time.time()
print('this is end_time ',end_time)

this is start_time  1536123240.1819103
this is end_time  1536123250.704512


In [3]:
from PIL import Image, ImageFilter

kitten = Image.open("screen.png")
blurrykitten = kitten.filter(ImageFilter.GaussianBlur)
blurrykitten.save("lulala.png")
blurrykitten.show()

In [ ]:
# 葛松 朱凯丰 黄君瑜 饶鸿斌 张佐贤 倪承煜 彭凯杰 吕东升 王昊远 彭健 范唯 吴迎楠 刘毅航 李慧俊 陈鹏